This is a Jupyter Notebook containing all kinds of tests for the different implementations of Flow formulations for the Steiner Tree Problem

In [1]:
import sys, os
sys.path.append(os.path.abspath('..'))
from graphilp.imports import ilpgraph, graph_formats
from graphilp.imports import networkx as imp_nx
import pcstModel
import pcst_flow_mtz
import pcst_flow_mtz_L
import pcst_flow_mtz_L_plus
import pcst_flow_rolf
import networkx as nx
import pcstModel
from gurobipy import *
import xlsxwriter
import random
import matplotlib.pyplot as plt
import sys, os
from datetime import datetime
%load_ext autoreload
%autoreload 2
foundTime = datetime.now()
oldSolution = 0

Helper function to reach comparable results with the ones from Ivana Ljubic in her 2006 Paper "An Algorithmic Framework for the Exact Solution of the Prize-Collecting Steiner Tree Problem".
While our goal is to always minimize costs, the objective function in her paper is to minimize opportunity costs. Since Ivana Ljubic and the results of this paper pose good results for comparison of our own algorithms, I rewrote my solution to match the one from the paper. This way our algorithms could be checked for functionality.

In [14]:
def computeMinimizationResult(sum_of_prizes, G, solution):
    result = sum_of_prizes
    res_nodes = set()
    for (u,v) in solution:
        result += G.get_edge_data(u, v)['weight']
        res_nodes.add(u)
        res_nodes.add(v)
    for u in G.nodes():
        if u in res_nodes:
            result -= G.nodes[u]['prize']
    print(result)
    return result

A callback function to make tracking of the computation time easier. Whenever a new incumbent solution is found, the timestamp is saved. In the end the time until the solution was found is calculated.

In [10]:
def callback(model, where):
    global foundTime, oldSolution
    
    if where == GRB.Callback.MIPSOL:
        newSolution = model.cbGet(GRB.Callback.MIPSOL_OBJBST)
        if newSolution > oldSolution:
            oldSolution = newSolution
            foundTime = datetime.now().replace(microsecond=0)

Tests for chosen models

In [17]:
# The input file
filePath = os.path.join(os.getcwd(), 'testInstances', 'PCST', 'i105M3.stp')

# Extracting the Graph, its terminals and its root from the input file
G, terminals, root = graph_formats.rooted_pcst_to_networkx(filePath)

# Transforming the Graph into an ILP Graph
Graph = imp_nx.read(G)

# Activate any of the below codelines and deactivate the currently active one to check different models.

newModel = pcstModel.pcstModel(Graph, terminals)
#newModel = pcst_flow_mtz.pcstModelMtz(Graph, terminals)
#newModel = pcst_flow_mtz_L.pcstModelMtzL(Graph, terminals)
#newModel = pcst_flow_mtz_L_plus.pcstModelMtzLPlus(Graph, terminals)

model = newModel.m

# Setting Starttime
startTime = datetime.now().replace(microsecond=0)

# Setting Parameters
model.Params.Threads = 4
model.Params.TimeLimit = 10
model.optimize()
variables = newModel.extractSolution(Graph)
# Extract the sum of all prizes from the Graph. This is needed for the reformulation of the solution value.
prizes = sum([G.nodes[node].get('prize', 0) for node in G.nodes()])
result = computeMinimizationResult(prizes, G, variables)

vars = []
flows = []
#  Extract edge variables
for k, v in newModel.G.edge_variables.items():
    if v.X > 0.5:
        vars.append(v)

# Extract solution Edges
solutionEdges = [newModel.var2edge[var] for var in vars if (var in newModel.var2edge)]
print(solutionEdges)

nodeSizes = []
# Build a Graph from the solution edges.
solutionGraph = nx.DiGraph()
solutionGraph.add_edges_from(solutionEdges)

Changed value of parameter Threads to 4
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 10.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 25925 rows, 26666 columns and 75445 nonzeros
Model fingerprint: 0xa3ced03b
Variable types: 0 continuous, 26666 integer (13333 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 6604 rows and 1799 columns
Presolve time: 0.16s
Presolved: 19321 rows, 24867 columns, 62065 nonzeros
Variable types: 0 continuous, 24867 integer (12449 binary)
Found heuristic solution: objective -13.0000000

Root relaxation: objective 2.660181e+05, 8828 iterations, 0.85 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Un

Benchmark Notebook for the PCST Test Files. This Notebook reads a set of input files, calculates their optimal objective function value and returns the solution in a simple Excel Spreadsheet.

# Benchmark Optimization Model

In [ ]:
def __write_solution(solution:list):
    
    # Encode the Solution in an Excel Worksheeet. The name of the Worksheet can be chosen arbitrarily.
    workbook = xlsxwriter.Workbook('Benchmarks_PCST_RolfVsMtz2.xlsx')
    worksheet = workbook.add_worksheet()
    bold = workbook.add_format({'bold':True})
    solutionRows = len(solution)
    solutionColumns = len(solution[0])
    
    # Headlines are written in boldface
    worksheet.write('A1', 'Instance', bold)
    worksheet.write('B1', 'Edges', bold)
    worksheet.write('C1', 'Nodes', bold)
    worksheet.write('D1', 'Terminals', bold)
    worksheet.write('E1', 'Runtime', bold)
    worksheet.write('F1', 'Objective Value', bold)
    worksheet.write('G1', 'Constraints', bold)
    worksheet.write('H1', 'MIP Gap', bold)
    worksheet.write('I1', 'Method', bold)
    worksheet.write('J1', 'Time2Sol', bold)
    
    # Write the solution to the Excel spreadsheet. The whole solution has to be written at once since xlsxwriter 
    # has a problem appending solutions.
    for i in range(solutionRows):
        for j in range(solutionColumns):
            worksheet.write(i + 1, j, solution[i][j])
    
        
def runBenchmark():
    # Array where Solutions are saved in
    solutions = []
    # List of Input files
    toTest = ['i101M2.stp', 'i101M3.stp', 'i105M2.stp', 'i105M3.stp']
    i = 0
    # Creating the basepath for all the files, i.e. specifying the folder the files are stored in.
    basepath = os.path.join('testInstances', 'PCST')
    # The base setting is to include all files from the whole directory. 
    for fname in os.listdir(basepath):
        # Excluding files which are not in our array of files which we want to test.
        if fname not in toTest:
            continue
        # Get the path of the file
        path = os.path.join(os.getcwd(), basepath, fname)
        # Skipping directories
        if os.path.isdir(path):
            continue
        # Open the input file and follow standard procedure for the tests
        with open(path) as f:
            print(path)
            # Read file
            G, terminals, root = graph_formats.rooted_pcst_to_networkx(path)
            Graph = imp_nx.read(G)
            pos = dict()
            pos[0] = (0,0)
            # Creating coordinates for each Node. Preparation for plots later on.
            for node in G.nodes(data=True):
                if 'coordinates' in node[1]:
                    pos[node[0]] = node[1]['coordinates']
                else:                    
                    pos[node[0]] = (random.randint(1,1000), random.randint(1,1000))
            # Setting capacity to a real high value. This way, capacity doesn't constrain the model.
            # We, however, have to set a capacity for the model to work.
            for edge in G.edges():
                G[edge[0]][edge[1]]['capacity'] = 1000

            print("Filename:", path)
            print("Connected: ", nx.is_connected(G))
            print("Nodes: ", len(G.nodes()))
            print("Edges: ", len(G.edges()))
            print("Root: ", root)
            print("Terminals: ", terminals)
            method = ""
            # Defines which model should be tested. Manual labor for every test run, not really well elaborated.
            for i in range(0,3):
                if i == 0:
                   model, var2edge = pcst_flow_mtz.pcstModelMtz(Graph, terminals)
                   method = "MTZ"
                if i == 1:
                    model, var2edge = pcst_flow_mtz_L.pcstModelMtzL(Graph, terminals)
                    startTime = datetime.now().replace(microsecond=0)
                    method = "MTZ-L"
                elif i == 2:
                    model, var2edge = pcstModel.pcstModel(Graph, terminals)
                    method = "Standard"
                elif i == 2:
                    model = pcst_flow_rolf.create_model(Graph, [root])
                    method = "Rolf PCST"
                # Parameter setting
                model.params.Threads = 2
                model.params.TimeLimit = 1200
                model.params.Seed = 1860
                model.optimize(callback)
                
                # Finding whole computation time
                timeElapsed = startTime - foundTime
                
                # Calculating comparable objective function
                solution = pcst_flow.extractSolution(Graph, model)
                prizes = sum([G.nodes[node].get('prize', 0) for node in G.nodes()])
                result = computeMinimizationResult(prizes, G, solution)

                solutions.append((path, len(G.edges()), len(G.nodes()), len(terminals), model.runTime, result,\
                  model.numConstrs, model.MIPGap, method, timeElapsed))

    __write_solution(solutions)         

if __name__ == '__main__':
    runBenchmark()

/home/tsauter/FlowStuff/testInstances/PCST/i105M3.stp
Filename: /home/tsauter/FlowStuff/testInstances/PCST/i105M3.stp
Connected:  True
Nodes:  741
Edges:  6296
Root:  1
Terminals:  [738, 739, 740]
Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0
Changed value of parameter TimeLimit to 1200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Seed to 1860
   Prev: 0  Min: 0  Max: 2000000000  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 2 threads
Optimize a model with 66035 rows, 26666 columns and 219143 nonzeros
Model fingerprint: 0x483321ea
Variable types: 0 continuous, 26666 integer (13333 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [1e+00, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+05]
Found heuristic solution: objective -0.0000000
Presolve removed 20981 rows and 745 colum

     0     0 177450.778    0  142 154392.848 177450.778  14.9%     -   29s
     0     0 177448.873    0  147 154392.848 177448.873  14.9%     -   29s
     0     0 177394.086    0  170 154392.848 177394.086  14.9%     -   30s
     0     0 177387.269    0  169 154392.848 177387.269  14.9%     -   30s
     0     0 177384.235    0  172 154392.848 177384.235  14.9%     -   31s
     0     0 177318.653    0  168 154392.848 177318.653  14.8%     -   31s
     0     0 177309.442    0  177 154392.848 177309.442  14.8%     -   32s
     0     0 177302.657    0  181 154392.848 177302.657  14.8%     -   32s
     0     0 177301.884    0  183 154392.848 177301.884  14.8%     -   32s
     0     0 177265.283    0  168 154392.848 177265.283  14.8%     -   33s
     0     0 177265.283    0  165 154392.848 177265.283  14.8%     -   38s
     0     2 176980.283    0  165 154392.848 176980.283  14.6%     -   41s
     7    10 176977.032    3  131 154392.848 176980.283  14.6%   393   45s
    27    30 176879.444  

KeyboardInterrupt: 